# An example of jupyter notebook + selenium for responsive bot  
## mimicking user operation (responsive operation & easy to develop)

##  Run all the blocks from top to bottom according to its order
### Game Website: https://www.travian.com/tw

In [1]:
#credentials
SERVER_URL = 'https://ts2.x1.arabics.travian.com/'
LOGIN_USERNAME = ''
LOGIN_PASSWORD = ''



VILLAGE_URL = SERVER_URL + 'dorf1.php'
TOWN_URL = SERVER_URL + 'dorf2.php'

Setting connection:

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time
import random
import re

 
options = Options()
options.add_argument("--disable-notifications")
 
driver = webdriver.Firefox()
driver.get(SERVER_URL+"login.php")
driver.implicitly_wait(10)

Login:

In [3]:
# login
write = driver.find_element_by_name("name");
write.clear();
write.send_keys(LOGIN_USERNAME);
write = driver.find_element_by_name("password");
write.clear();
write.send_keys(LOGIN_PASSWORD);
driver.find_element_by_id("s1").click();

Function for checking if there's an upgradable resource:

In [4]:
def Upgradable_Resources_Check(driver):
    # Resources Check
    RFC = driver.find_element_by_id("resourceFieldContainer");
    #print(RFC.getText());

    #a = driver.find_element_by_tag_name("a");
    #a = driver.find_element_by_cssSelector("div.*");
    #RFC = driver.find_elements_by_css_selector("div[id=resourceFieldContainer] > a  ")
    #RFC = driver.find_elements_by_css_selector("div[id*=Fiel] > a  ")
    #RFC = driver.find_elements_by_css_selector("div[class*=Field][class*=tribe3] > a  ")

    RFC = driver.find_elements_by_css_selector("a[class*=good][class*=level]  ")  ## <--- use this

    # 1:Woodcutter 2:Clay Pit 3:Iron Mine 4:Cropland
    Upgradable_Resources = [[],[],[],[],[]];

    for x in RFC:
        if x.text == "":
           continue;

        class_text = x.get_attribute("class");
        level = int(x.text);
        #print(class_text);
        #print(x.get_attribute("href"));
        #print(level);

        class_text = x.get_attribute("class");

        cat_num = int(re.search(r'^.*gid(\d).*$',class_text).group(1));
        #print("cat_num = ", cat_num)

        Upgradable_Resources[cat_num].append((level,x));

        #print("---------")

    for i in range(1,5):
        Upgradable_Resources[i].sort(key=lambda y: y[0]);


    #Next_Upgrade_x = None;
    No_Next_Upgrade = True;
    for x in Upgradable_Resources:
        if x == []:
            continue;
        #print(x)
        No_Next_Upgrade = False;

    if(No_Next_Upgrade):
        return [-1,Upgradable_Resources];

    #Production per hour
    Production_per_hour = {};
    Production_Table_name = driver.find_elements_by_css_selector("table[id=production] > tbody > tr > td[class*=res] ")
    Production_Table_val = driver.find_elements_by_css_selector("table[id=production] > tbody > tr > td[class*=num] ")
    for i in range(4):
        name = Production_Table_name[i].text.split(':')[0];
        val = Production_Table_val[i].text;
        Production_per_hour[name] = int(val.encode('ascii',"ignore"));
    #print(Production_per_hour)

    ## production_sum_Weight
    weighted_production = Production_per_hour;
    weighted_production['Crop'] += 200;
    weighted_production['Iron'] += 80;
    weighted_production['Lumber'] += 40;
    weighted_production['Clay'];


    weighted_production = sorted(weighted_production.items(), key=lambda x:x[1]);
    weighted_production = dict(weighted_production);
    print(weighted_production);

    Resources_name_val_dict = {"Lumber":1, "Clay":2, "Iron":3, "Crop":4};
    ###Next_upgrade_resources_name = list(weighted_production.keys())[0];
    for name in weighted_production.keys():
        val = Resources_name_val_dict[name]; 
        print(name, val);
        if(Upgradable_Resources[val] == []):
            continue;
        
        Next_upgrade_resources_name = name;
        Next_upgrade_resources_val = val;
        break;

    #print(Resources_name_val_dict);
    #print(Next_upgrade_resources_name,Next_upgrade_resources_val);

    return [Next_upgrade_resources_val,Upgradable_Resources];


An example call for upgradable resource check: 

In [5]:
result = Upgradable_Resources_Check(driver);
Next_upgrade_resources_val = result[0];
Upgradable_Resources = result[1];
#print(Next_upgrade_resources_val);
#print(Upgradable_Resources);

Choose a resource field to upgrade (only upgrade when Next_upgrade_resources_val={1,2,3,4}):

In [6]:
 def Upgrade_Resource(driver,Next_upgrade_resources_val,Upgradable_Resources):
    if(Next_upgrade_resources_val < 1 or Next_upgrade_resources_val > 4) :
        return;
    
    ###level_x_tuple = Upgradable_Resources[3][0]; #Iron
    level_x_tuple = Upgradable_Resources[Next_upgrade_resources_val][0];
    #print(x_tuple[0], x_tuple[1])
    level_x_tuple[1].click();

    BuildButton = driver.find_element_by_css_selector("button[class*=textButtonV1][class*=green][class*=build]");
    #print(BuildButton);
    BuildButton.click();
    
    return


An example call to upgrade a resource (only upgrade when Next_upgrade_resources_val={1,2,3,4}): 

In [7]:
Upgrade_Resource(driver,-1,[]);

Continously checking & upgrading resource field without logout and relogin:

In [9]:
##Execute as a Loop (without logout)
#while(1):
#    result = Upgradable_Resources_Check(driver);
#    Next_upgrade_resources_val = result[0];
#    Upgradable_Resources = result[1];
#    #print(Next_upgrade_resources_val, Upgradable_Resources);
#    if(Next_upgrade_resources_val == -1) :
#        continue;
#
#    Upgrade_Resource(driver, Next_upgrade_resources_val, Upgradable_Resources)
#    
#    base_sleep_time = 3500;
#   #base_sleep_time = 1;
#    rand_sleep_time = random.randint(1,100);
#    sleep_time = base_sleep_time + rand_sleep_time;
#   print(sleep_time);
#    time.sleep(sleep_time);

Logout Website:

In [8]:
#leave village
driver.find_element_by_css_selector("li[class*=logout] > a").click();

Continously checking & upgrading resource field with logout and relogin:

In [9]:
#Execute as a Loop (logout + login)
#counter = 1;
while(1):
#while(counter):
    #counter -= 1;
    try:
        prompt = driver.find_element_by_css_selector("a[class*=cmpboxbtn][class*=cmpboxbtnyes]  ")  ## <--- use this
    except:
        print("No prompt.");
    else:
        print(prompt);
        prompt.click();
    finally:
        driver.implicitly_wait(10)
    
    # login
    write = driver.find_element_by_name("name");
    write.clear();
    write.send_keys(LOGIN_USERNAME);
    write = driver.find_element_by_name("password");
    write.clear();
    write.send_keys(LOGIN_PASSWORD);
    driver.find_element_by_id("reloginButton").click();

    result = Upgradable_Resources_Check(driver);
    Next_upgrade_resources_val = result[0];
    Upgradable_Resources = result[1];
    #print(Next_upgrade_resources_val, Upgradable_Resources);
    if(Next_upgrade_resources_val != -1) :
        Upgrade_Resource(driver, Next_upgrade_resources_val, Upgradable_Resources);
        driver.implicitly_wait(10)

    #leave village
    driver.find_element_by_css_selector("li[class*=logout] > a").click();
    driver.implicitly_wait(10)

    base_sleep_time = 3500;
    #base_sleep_time = 1;
    rand_sleep_time = random.randint(1,100);
    sleep_time = base_sleep_time + rand_sleep_time;
    print(sleep_time);
    time.sleep(sleep_time);

No prompt.
3528
No prompt.
3533
No prompt.
3559
No prompt.
3552
No prompt.
3556
No prompt.
{'Clay': 1001, 'Iron': 1032, 'Lumber': 1041, 'Crop': 1083}
Clay 2
3597
No prompt.
3540
No prompt.
3586
No prompt.
3542
No prompt.
3530
No prompt.
3552
<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="5723f827-88ef-4fce-9138-88d3065c81f7", element="01f8a33d-a248-408f-b45d-f30691daa111")>
{'Iron': 1032, 'Lumber': 1041, 'Clay': 1064, 'Crop': 1075}
Iron 3
Lumber 1
3527
No prompt.
3502
No prompt.
3524
No prompt.
3507
No prompt.
3513
No prompt.
3579
No prompt.
{'Iron': 1032, 'Clay': 1064, 'Crop': 1073, 'Lumber': 1104}
Iron 3
3545
No prompt.
3523
No prompt.
3566
No prompt.
3504
No prompt.
3546
No prompt.
3543
No prompt.
3565
No prompt.
3533
No prompt.
3532
No prompt.
{'Crop': 1058, 'Clay': 1064, 'Lumber': 1104, 'Iron': 1111}
Crop 4
3553
No prompt.
3508
No prompt.
3577
No prompt.
3546
No prompt.
3542
No prompt.
{'Clay': 1064, 'Lumber': 1104, 'Iron': 1111, 'Crop': 1140}
Clay 2
Lumber 1
I

NoSuchElementException: Message: Unable to locate element: li[class*=logout] > a
